In [12]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy

#import packages
import pandas as pd
import numpy as np
import datetime as dt

In [24]:
# Import data
data_crime = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/crime_data_edited.csv')
data_share = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/election_data_edited.csv')

In [25]:
data_share[data_share['city'] == 'Meissen']

,Unnamed: 0,city,party,Anzahl Bewerber,Prozent Frauen,darunter Frauen,year,state
13649,13649,Meissen,CDU,16.0,18.750000,3.0,2014,SN
13650,13650,Meissen,DIE LINKE,17.0,23.529412,4.0,2014,SN
13651,13651,Meissen,FDP,15.0,6.666667,1.0,2014,SN
13652,13652,Meissen,SPD,11.0,27.272727,3.0,2014,SN
13653,13653,Meissen,GRÜNE,4.0,25.000000,1.0,2014,SN
14928,14928,Meissen,CDU,25.0,28.000000,7.0,2019,SN
14929,14929,Meissen,DIE LINKE,14.0,42.857143,6.0,2019,SN
14930,14930,Meissen,SPD,10.0,60.000000,6.0,2019,SN
14931,14931,Meissen,FDP,18.0,22.222222,4.0,2019,SN
14932,14932,Meissen,AfD,9.0,11.111111,1.0,2019,SN


In [26]:
# get into datetime format
data_crime['year'] = data_crime['date'].astype(str).str.split('.', expand=True)[2]
data_crime['month'] = data_crime['date'].astype(str).str.split('.', expand=True)[1]
data_crime['day'] = data_crime['date'].astype(str).str.split('.', expand=True)[0]
data_crime['date'] = pd.to_datetime(data_crime[['year', 'month', 'day']].astype(float), errors='ignore')

In [27]:
# Merge crime and candidate statistics with merge over city string
data_crime = data_crime[(data_crime['Land'] == 14) | (data_crime['Land'] == 8) | (data_crime['Land'] == 16)]
data_share['city'] = data_share['city'].str.replace(r'Stadt ', '')
data_share['city'] = data_share['city'].str.replace('Ä', 'A').str.replace('Ö', 'O').str.replace('Ü', 'U')

list_cities_share = data_share.sort_values('city')['city'].unique()
list_cities_crime = data_crime['city'].unique()

data_crime.rename({'Land':'state'}, axis=1, inplace=True)
data_crime['state'] = data_crime['state'].map({16:'TH', 8:'BW', 14:'SN'})

In [28]:
data_share.rename({'year':'cycle_1'}, axis=1, inplace=True)
data_share['cycle_2'] = data_share['cycle_1']
data_share['cycle_3'] = data_share['cycle_1']

Merging over years and cities? How many different cutoffs? Directly on election day, one month before and three months before

Cut-off days for the three states: 05.26.2019, 04.26.2019, 02.26.2019

In [29]:
# creating different cycle variables in crime data to merge over
data_crime['cycle_1'] = 2014
data_crime.loc[data_crime['date'] > dt.datetime(2019, 5, 26, 0, 0), 'cycle_1'] = 2019

data_crime['cycle_2'] = 2014
data_crime.loc[data_crime['date'] > dt.datetime(2019, 4, 26, 0, 0), 'cycle_2'] = 2019

data_crime['cycle_3'] = 2014
data_crime.loc[data_crime['date'] > dt.datetime(2019, 2, 26, 0, 0), 'cycle_3'] = 2019

In [30]:
list_double = pd.DataFrame()
for city in data_share['city'].unique():
    
    data_city = data_share[data_share['city'] == city]
    for party in data_city['party'].unique():
        
        data_party = data_city[data_city['party'] == party]
        for election in data_party['cycle_1'].unique():
            
            data_election = data_party[data_party['cycle_1'] == election]
            
            if len(data_election) > 1:
                
                # append to dataset of 
                list_double = list_double.append(data_election)
                

In [36]:
# drop cities that are not uniquely identifable
data_share = data_share[~data_share.city.isin(list_double.city)]

In [40]:
data_base_1 = pd.merge(data_crime, data_share, on=['cycle_1', 'city', 'party', 'state'], how='right', validate='m:1')
data_base_2 = pd.merge(data_crime, data_share, on=['cycle_2', 'city', 'party', 'state'], how='right', validate='m:1')
data_base_3 = pd.merge(data_crime, data_share, on=['cycle_3', 'city', 'party', 'state'], how='right', validate='m:1')

In [51]:
data_base_1 = data_base_1[data_base_1.columns[(~data_base_1.columns.str.contains(r'_x')) & (~data_base_1.columns.str.contains(r'_y'))]]
data_base_2 = data_base_2[data_base_2.columns[(~data_base_2.columns.str.contains(r'_x')) & (~data_base_2.columns.str.contains(r'_y'))]]
data_base_3 = data_base_3[data_base_3.columns[(~data_base_3.columns.str.contains(r'_x')) & (~data_base_3.columns.str.contains(r'_y'))]]

In [56]:
data_base_1 = data_base_1[['crime', 'city', 'law', 'date', 'background', 'suspects', 'party', 'state', 'year', 'month', 'day', 'cycle_1', 'Anzahl Bewerber', 'Prozent Frauen', 'darunter Frauen']]

In [57]:
data_base_1

,crime,city,law,date,background,suspects,party,state,year,month,day,cycle_1,Anzahl Bewerber,Prozent Frauen,darunter Frauen
0,Sachbeschadigung,Stuttgart,STGB-303,2019-07-25,Links,0.0,AfD,BW,2019,07,25,2019,60.0,26.666667,16.0
1,Verleumdung ohne Verleumdung ohne sexuelle Gru...,Stuttgart,STGB-187,2019-07-25,Links,0.0,AfD,BW,2019,07,25,2019,60.0,26.666667,16.0
2,Volksverhetzung,Stuttgart,STGB-130,2019-08-12,Rechts,1.0,AfD,BW,2019,08,12,2019,60.0,26.666667,16.0
3,Sachbeschadigung,Stuttgart,STGB-303,2019-08-26,Links,0.0,AfD,BW,2019,08,26,2019,60.0,26.666667,16.0
4,Sachbeschadigung,Stuttgart,STGB-303,2019-07-25,Links,0.0,AfD,BW,2019,07,25,2019,60.0,26.666667,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18176,NaN,Neuhaus am Rennweg,NaN,NaT,NaN,NaN,GRÜNE,TH,NaN,NaN,NaN,2014,1.0,100.000000,1.0
18177,NaN,Sonneberg,NaN,NaT,NaN,NaN,GRÜNE,TH,NaN,NaN,NaN,2014,7.0,14.285714,1.0
18178,NaN,Rudolstadt,NaN,NaT,NaN,NaN,GRÜNE,TH,NaN,NaN,NaN,2014,10.0,40.000000,4.0
18179,NaN,Saalfeld/Saale,NaN,NaT,NaN,NaN,GRÜNE,TH,NaN,NaN,NaN,2014,13.0,38.461538,5.0
